In [15]:
import base64
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Optional

# Initialize OpenAI client
client = OpenAI()

# Define your Pydantic schema for the form fields
class FormData(BaseModel):
    """Schema for extracting health-related form fields from the image"""
    smokes: bool = Field(description="Whether the person smokes (Yes/No, True/False)")
    cigarettes_per_day: Optional[int] = Field(None, description="Number of cigarettes smoked per day. None if doesn't smoke or not specified")
    height_cm: Optional[float] = Field(None, description="Height in centimeters")
    weight_kg: Optional[float] = Field(None, description="Weight in kilograms")
    date_of_birth: Optional[str] = Field(None, description="The birth year of the person in format DD.MM.YYYY")

def encode_image(image_path: str) -> str:
    """Encode image to base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def extract_form_fields(image_path: str) -> FormData:
    """
    Extract form fields from a JPEG image using OpenAI's structured outputs
    
    Args:
        image_path: Path to the JPEG form image
        
    Returns:
        FormData object with extracted fields
    """
    # Encode the image
    base64_image = encode_image(image_path)
    
    # Make API call with structured output
    response = client.responses.parse(
        model="gpt-4o",  # Vision-capable model
        input=[
            {
                "role": "system",
                "content": "You are a medical form extraction assistant. Extract health-related information from the form. For height and weight, convert to centimeters and kilograms if given in other units. If smoking status is 'No' or unchecked, set cigarettes_per_day to None."
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "input_text",
                        "text": "Please extract all the form fields from this image."
                    },
                    {
                        "type": "input_image",
                        "image_url": f"data:image/jpeg;base64,{base64_image}"
                    }
                ]
            }
        ],
        text_format=FormData
    )
    
    # Return parsed structured data
    return response.output_parsed

In [16]:

# Replace with your image path
image_path = "assets/pax_form_filled_1.jpeg"

# Extract form fields
form_data = extract_form_fields(image_path)

print(form_data.model_dump())

{'smokes': True, 'cigarettes_per_day': 5, 'height_cm': 170.0, 'weight_kg': 80.0, 'date_of_birth': '12.11.1983'}


In [22]:
import requests

# The URL for the POST request
url = "http://localhost:8000/process"

# The path to the file you want to upload
file_path = "assets/pax_form_filled_1.jpeg"

# The data fields to be sent along with the file
# In your curl command, this is '-F "type=image"'
data = {
    "type": "image"
}

# The files dictionary, where the key is the field name ('file' in this case)
# and the value is a tuple: (filename, file_object or file_contents, content_type)
# If only providing the file object, it should be a tuple (filename, file_object)
try:
    with open(file_path, "rb") as f:
        files = {
            "file": (file_path, f, "image/jpeg") # You can usually omit the content type, requests will infer it.
            # A simpler, common structure is:
            # "file": f
        }

        # Send the POST request with data and files
        response = requests.post(url, data=data, files=files)

    # Check the response
    print(f"Status Code: {response.status_code}")
    print("Response Body:")
    print(response.text)

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Make sure it exists in the current directory.")
except requests.exceptions.RequestException as e:
    print(f"An error occurred during the request: {e}")

Status Code: 200
Response Body:
{"status":"success","source":"image_extraction","data":{"smokes":true,"cigarettes_per_day":5,"height_cm":170.0,"weight_kg":80.0,"date_of_birth":"12.11.1983"}}
